In [2]:
import glob
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


%matplotlib inline

plt.rcParams["figure.figsize"] = [16, 10]

## Normalize JSON turnout data

https://vt.ncsbe.gov/RegStat/

In [4]:
ls -lah data/registration

total 1456
drwxr-xr-x  28 mookerji  staff   896B Aug  7 18:43 ./
drwxr-xr-x   4 mookerji  staff   128B Aug  7 18:42 ../
-rw-r--r--   1 mookerji  staff    26K Aug  7 18:35 02-25-2020.json
-rw-r--r--   1 mookerji  staff    26K Aug  7 18:34 02-29-2020.json
-rw-r--r--   1 mookerji  staff    26K Jul 27 22:11 03-03-2020.json
-rw-r--r--   1 mookerji  staff    26K Jul 27 22:11 03-07-2020.json
-rw-r--r--   1 mookerji  staff    26K Jul 27 22:10 03-14-2020.json
-rw-r--r--   1 mookerji  staff    26K Jul 27 22:09 03-21-2020.json
-rw-r--r--   1 mookerji  staff    26K Jul 27 22:08 03-28-2020.json
-rw-r--r--   1 mookerji  staff    27K Jul 27 22:07 04-04-2020.json
-rw-r--r--   1 mookerji  staff    27K Jul 27 22:06 04-11-2020.json
-rw-r--r--   1 mookerji  staff    27K Jul 27 22:06 04-18-2020.json
-rw-r--r--@  1 mookerji  staff    27K Jul 27 21:18 04-25-2020.json
-rw-r--r--@  1 mookerji  staff    27K Jul 27 21:18 05-02-2020.json
-rw-r--r--@  1 mookerji  staff    27K Jul 27 21:18 05-09-2020.json
-rw-r--r-

In [7]:
glob.glob('data/registration/*.json')

['data/registration/03-14-2020.json',
 'data/registration/03-21-2020.json',
 'data/registration/08-01-2020.json',
 'data/registration/07-11-2020.json',
 'data/registration/04-18-2020.json',
 'data/registration/07-25-2020.json',
 'data/registration/06-23-2020.json',
 'data/registration/07-04-2020.json',
 'data/registration/03-07-2020.json',
 'data/registration/02-25-2020.json',
 'data/registration/06-13-2020.json',
 'data/registration/07-18-2020.json',
 'data/registration/05-09-2020.json',
 'data/registration/05-16-2020.json',
 'data/registration/04-04-2020.json',
 'data/registration/05-02-2020.json',
 'data/registration/04-25-2020.json',
 'data/registration/06-20-2020.json',
 'data/registration/05-23-2020.json',
 'data/registration/03-28-2020.json',
 'data/registration/05-30-2020.json',
 'data/registration/06-27-2020.json',
 'data/registration/03-03-2020.json',
 'data/registration/06-06-2020.json',
 'data/registration/02-29-2020.json',
 'data/registration/04-11-2020.json']

In [21]:
def registrations_to_csv():
    files = glob.glob('data/registration/*.json')
    data = []
    for f in files:
        df = pd.read_json(f)
        df['date'] = pd.to_datetime(f.split('/')[2].rstrip('.json'))
        data.append(df)
    return pd.concat(data)

In [30]:
registrations = registrations_to_csv()
registrations.head()

,CountyName,Democrats,Republicans,Libertarians,Green,Constitution,Unaffiliated,White,Black,AmericanIndian,Other,Hispanic,Male,Female,UnDisclosedGender,Total,date
0,ALAMANCE,36792,33375,556,17,42,31955,70181,21076,258,11222,3998,44020,53978,4739,102737,2020-03-14
1,ALEXANDER,5242,10930,91,1,15,7518,21612,931,35,1219,334,11145,12006,646,23797,2020-03-14
2,ALLEGHANY,2194,2704,33,2,1,2259,6666,73,7,447,177,3330,3606,257,7193,2020-03-14
3,ANSON,10111,2478,29,7,24,3372,6656,6818,27,2520,82,6205,7558,2258,16021,2020-03-14
4,ASHE,4539,8082,76,3,7,5660,17504,103,21,739,208,8491,9511,365,18367,2020-03-14


In [23]:
registrations.to_csv('data/registrations2020.csv', index=False)

## CVAP (estimate)

https://data.census.gov/cedsci/table?q=B05003&g=0400000US37,37.050000&tid=ACSDT5Y2018.B05003&vintage=2014&y=2018

In [34]:
cols = ['Estimate!!Total!!Female!!18 years and over!!Foreign born!!Naturalized U.S. citizen',
        'Estimate!!Total!!Female!!18 years and over!!Native',
        'Estimate!!Total!!Male!!18 years and over!!Foreign born!!Naturalized U.S. citizen',
        'Estimate!!Total!!Male!!18 years and over!!Native',
       ] 

def read_cvap_data():
    cvap = 'data/cvap/productDownload_2020-07-28T211825/ACSDT5Y2018.B05003_data_with_overlays_2020-07-28T211821.csv'
    cvap_df = pd.read_csv(cvap, skiprows=1)
    cvap_df['CountyName'] = cvap_df['Geographic Area Name'].apply(lambda t: t.replace(' County, North Carolina', '').upper())
    cvap_df.set_index('CountyName', inplace=True, drop=False)
    cvap_df['cvap_all_races'] = cvap_df[cols].sum(axis=1)
    cvap_df.drop('NORTH CAROLINA', inplace=True)
    return cvap_df[['cvap_all_races', 'CountyName']]

In [35]:
cvap_df = read_cvap_data()
cvap_df

,cvap_all_races,CountyName
CountyName,,
MITCHELL,11930,MITCHELL
GREENE,15373,GREENE
CHOWAN,11146,CHOWAN
CALDWELL,63099,CALDWELL
CATAWBA,115407,CATAWBA
...,...,...
DAVIDSON,122830,DAVIDSON
HERTFORD,18989,HERTFORD
PENDER,44201,PENDER


In [37]:
cvap_df.to_csv('data/cvap2020_all_races.csv', index=False)

## Post-processed

In [46]:
def add_cvap(registrations, cvap):
    joined = registrations.join(cvap[['cvap_all_races', 'CountyName']], on='CountyName', rsuffix='_r').drop(labels='CountyName_r', axis=1)
    for c in ['Democrats', 'Republicans', 'Libertarians', 'Green', 'Constitution', 'Unaffiliated', 'Total']:
        joined[c+'_pct'] = 100*joined[c]/joined['cvap_all_races']
    return joined

In [49]:
add_cvap(registrations, cvap_df)

,CountyName,Democrats,Republicans,Libertarians,Green,Constitution,Unaffiliated,White,Black,AmericanIndian,...,Total,date,cvap_all_races,Democrats_pct,Republicans_pct,Libertarians_pct,Green_pct,Constitution_pct,Unaffiliated_pct,Total_pct
0,ALAMANCE,36792,33375,556,17,42,31955,70181,21076,258,...,102737,2020-03-14,115973,31.724626,28.778250,0.479422,0.014659,0.036215,27.553827,88.586999
1,ALEXANDER,5242,10930,91,1,15,7518,21612,931,35,...,23797,2020-03-14,28629,18.310105,38.178071,0.317860,0.003493,0.052394,26.260086,83.122009
2,ALLEGHANY,2194,2704,33,2,1,2259,6666,73,7,...,7193,2020-03-14,8638,25.399398,31.303542,0.382033,0.023154,0.011577,26.151887,83.271591
3,ANSON,10111,2478,29,7,24,3372,6656,6818,27,...,16021,2020-03-14,19935,50.719839,12.430399,0.145473,0.035114,0.120391,16.914974,80.366190
4,ASHE,4539,8082,76,3,7,5660,17504,103,21,...,18367,2020-03-14,21275,21.334900,37.988249,0.357227,0.014101,0.032902,26.603995,86.331375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,WAYNE,29395,23779,352,19,41,19199,41137,24187,138,...,72785,2020-04-11,87849,33.460825,27.068037,0.400688,0.021628,0.046671,21.854546,82.852394
96,WILKES,8262,21843,142,7,12,11168,37883,1624,44,...,41434,2020-04-11,52320,15.791284,41.748853,0.271407,0.013379,0.022936,21.345566,79.193425
97,WILSON,27525,13398,181,5,34,14208,27466,22954,103,...,55351,2020-04-11,59290,46.424355,22.597403,0.305279,0.008433,0.057345,23.963569,93.356384
98,YADKIN,3522,13233,98,9,10,6670,21237,736,28,...,23542,2020-04-11,28312,12.439955,46.739898,0.346143,0.031789,0.035321,23.558915,83.152020


In [50]:
joined.to_csv('data/joined2020_all_absolute.csv', index=False)

In [53]:
def compute_changes(joined):
    diffs = []
    for county, df in joined.groupby('CountyName'):
        dfg = df.sort_values('date').set_index('date').drop('CountyName', axis=1)
        dfh = dfg - dfg.iloc[0]
        dfh['CountyName'] = county
        dfh['cvap_all_races'] = dfg['cvap_all_races']
        diffs.append(dfh)
    return pd.concat(diffs)

In [55]:
changes = compute_changes(joined)
changes.head()

,Democrats,Republicans,Libertarians,Green,Constitution,Unaffiliated,White,Black,AmericanIndian,Other,...,Total,cvap_all_races,Democrats_pct,Republicans_pct,Libertarians_pct,Green_pct,Constitution_pct,Unaffiliated_pct,Total_pct,CountyName
date,,,,,,,,,,,,,,,,,,,,,
2020-02-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,115973,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ALAMANCE
2020-02-29,1.0,6.0,0.0,0.0,0.0,-22.0,6.0,-10.0,1.0,-12.0,...,-15.0,115973,0.000862,0.005174,0.000000,0.000000,0.000000,-0.018970,-0.012934,ALAMANCE
2020-03-03,58.0,23.0,0.0,0.0,0.0,1.0,70.0,7.0,1.0,4.0,...,82.0,115973,0.050012,0.019832,0.000000,0.000000,0.000000,0.000862,0.070706,ALAMANCE
2020-03-07,59.0,22.0,-1.0,-1.0,0.0,-3.0,66.0,6.0,1.0,3.0,...,76.0,115973,0.050874,0.018970,-0.000862,-0.000862,0.000000,-0.002587,0.065532,ALAMANCE
2020-03-14,113.0,36.0,-2.0,-1.0,-2.0,8.0,116.0,26.0,2.0,8.0,...,152.0,115973,0.097436,0.031042,-0.001725,-0.000862,-0.001725,0.006898,0.131065,ALAMANCE


In [56]:
changes.to_csv('data/joined2020_changes.csv', index=False)